In [0]:
!mkdir data
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/blackberry.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/blueberry.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/grapes.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pear.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/strawberry.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/watermelon.npy

In [0]:
!mv *npy data/
!ls data

In [0]:
from sklearn.model_selection import train_test_split as tts
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.layers.core import Activation
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.applications import mobilenet
from random import randint
import numpy as np
import os
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random

In [0]:
class DoodleCNN():
    def __init__(self, data_dir, limit=None, split=0.05):
        self.classes = {}
        self.load(data_dir, split, limit)
        self.buildModel()
        
    def load(self, data_dir, split=0.05, limit=None):
        X = []
        Y = []
        label = -1
        for file in os.listdir(data_dir):
            if file.endswith(".npy"):
                label += 1
                contents = self.reshape(np.load(data_dir + file)[0:limit])
                self.classes[label] = file[:-4]
                X = X + contents
                Y = Y + [label for _ in range(0, len(contents))]
        X = self.normalize(X)
        self.X_train, self.X_test, self.Y_train, self.Y_test = self.split(X, Y, split)
        
    def reshape(self, contents):
        reshaped = []
        for i in range(len(contents)):
            image = np.reshape(contents[i], (28, 28, 1))
            reshaped.append(image)
        return reshaped
        
    def normalize(self, data):
        return np.interp(data, [0, 255], [-1, 1])
      
    def denormalize(self, data):
        return np.interp(data, [-1, 1], [0, 255])
            
    def split(self, X, Y, split=0.05):
        X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=split)
        Y_train = np_utils.to_categorical(Y_train, len(self.classes))
        Y_test = np_utils.to_categorical(Y_test, len(self.classes))
        return X_train, X_test, Y_train, Y_test
    
    def buildModel(self):
        # should try batch normalization layer
        self.model = Sequential()
        self.model.add(Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=(28,28,1)))
        self.model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
        self.model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.5))
        self.model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(len(self.classes), activation='softmax'))
        # LeNet
        # self.model.add(Conv2D(20, (5,5), padding='same',input_shape = (28,28,1)))
        # self.model.add(Activation(activation = 'relu'))
        # self.model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        # self.model.add(Conv2D(50, (5,5), padding="same"))
        # self.model.add(Activation(activation = 'relu'))
        # self.model.add(MaxPooling2D(pool_size = (2, 2),strides = (2, 2)))
        # self.model.add(Flatten())
        # self.model.add(Dense(500))
        # self.model.add(Activation(activation = 'relu'))
        # self.model.add(Dense(len(self.classes)))
        # self.model.add(Activation("softmax"))
        
    def train(self, epochs=10):
        self.model.compile(loss='categorical_crossentropy', optimizer=SGD(lr = 0.01), metrics=['accuracy'])
        self.model.fit(np.array(self.X_train), np.array(self.Y_train), batch_size=32, epochs=epochs, shuffle=True)
        
    def test(self):
        loss, acc = self.model.evaluate(self.X_test, self.Y_test, verbose=0)
        print('\nTesting acc: {}, loss: {}\n'.format(acc, loss))
        
    def predict(self, arr):
        return self.model.predict(arr)
        
    def randomPrediction(self):
        index = random.randint(0,len(self.X_test) - 1)
        
        y_hat = self.predict(self.X_test)[index]
        pred_class = self.classes[max(range(len(y_hat)), key=y_hat.__getitem__)]
        
        y = self.Y_test[index]
        actual_class = self.classes[max(range(len(y)), key=y.__getitem__)]
        
        print("Predicition Probabilities:", y_hat)
        print("\nPredicted Class: {}\nActual Class: {}".format(pred_class, actual_class))
        
        img = self.denormalize(self.X_test[index])
        plot = plt.figure(figsize=(4,4))
        plt.imshow(img.reshape(28,28))
        plt.grid(False)


In [0]:
nn = DoodleCNN("data/", 5000, 0.1)
print(nn.classes)
print(np.shape(nn.X_train), np.shape(nn.Y_train), np.shape(nn.X_test), np.shape(nn.Y_test))

In [0]:
nn.model.summary()

In [0]:
nn.train(100)

In [0]:
nn.test()
nn.randomPrediction()